In [81]:
# %pip install cloudscraper

import pandas as pd
from bs4 import BeautifulSoup, Comment
import re
import cloudscraper
import time
import random
import csv
import numpy as np

SEASON_LIST=['2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024', '2025']
SEASON_URL = 'https://www.pro-football-reference.com/years/{}/games.htm'
GAME_URL = 'https://www.pro-football-reference.com/boxscores/{}.htm' # add game_id to the end of it
GAME_URL_LIST = []

In [5]:
# Create a scraper that bypasses Cloudflare
scraper = cloudscraper.create_scraper(
    browser={
        'browser': 'chrome',
        'platform': 'windows',
        'mobile': False
    }
)

game_id_list = []

for year in SEASON_LIST:
    url = SEASON_URL.format(year)
    # print(f"Processing {year}...")
    
    # Add random delay
    time.sleep(random.uniform(2, 4))
    
    try:
        response = scraper.get(url)
        # print(f"Status code: {response.status_code}")
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            # print(f"Successfully fetched {url}")
            
            # Find the table with id="games"
            games_table = soup.find('table', {'id': 'games'})
            
            if games_table:
                # print(f"Found games table for {year}")

                # Method 1: Exact text match
                boxscore_links = games_table.find_all('a', string='boxscore')
                
                print(f"Found {len(boxscore_links)} boxscore links for {year}")
                
                # Extract href attributes
                for link in boxscore_links:
                    href = link.get('href')
                    if href:
                        # print(f"Found href: {href}")

                        # 1. Get the last part after last slash
                        game_id = href.split('/')[-1]
                        
                        # 2. Remove file extension if present
                        if '.' in game_id:
                            game_id = game_id.split('.')[0]
                            
                        
                        game_id_list.append(game_id)
                        # print(f"  → Extracted game ID: {game_id}")
        else:
            print(f"Failed with status: {response.status_code}")
            
    except Exception as e:
        print(f"Error: {e}")

# print(len(game_id_list))
game_id_list

Found 267 boxscore links for 2016
Found 267 boxscore links for 2017
Found 267 boxscore links for 2018
Found 267 boxscore links for 2019
Found 269 boxscore links for 2020
Found 285 boxscore links for 2021
Found 284 boxscore links for 2022
Found 285 boxscore links for 2023
Found 285 boxscore links for 2024
Found 256 boxscore links for 2025


['201609080den',
 '201609110rav',
 '201609110jax',
 '201609110atl',
 '201609110htx',
 '201609110nyj',
 '201609110phi',
 '201609110kan',
 '201609110oti',
 '201609110nor',
 '201609110sea',
 '201609110clt',
 '201609110dal',
 '201609110crd',
 '201609120was',
 '201609120sfo',
 '201609150buf',
 '201609180pit',
 '201609180cle',
 '201609180det',
 '201609180htx',
 '201609180nwe',
 '201609180car',
 '201609180was',
 '201609180nyg',
 '201609180crd',
 '201609180ram',
 '201609180rai',
 '201609180den',
 '201609180sdg',
 '201609180min',
 '201609190chi',
 '201609220nwe',
 '201609250buf',
 '201609250car',
 '201609250cin',
 '201609250mia',
 '201609250gnb',
 '201609250jax',
 '201609250nyg',
 '201609250oti',
 '201609250tam',
 '201609250sea',
 '201609250clt',
 '201609250phi',
 '201609250kan',
 '201609250dal',
 '201609260nor',
 '201609290cin',
 '201610020jax',
 '201610020chi',
 '201610020nwe',
 '201610020htx',
 '201610020rav',
 '201610020nyj',
 '201610020atl',
 '201610020was',
 '201610020tam',
 '201610020crd

In [62]:
# get the qb stuff
def get_data(game_id_list):
   for game_id in game_id_list:

      failed_ids = []

      url = GAME_URL.format(game_id)
      print("Processing: " + url)

      # Add random delay
      time.sleep(random.uniform(2, 4))

      try:
         response = scraper.get(url)
         print(f"Status code: {response.status_code}")
         
         if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            print(f"Successfully fetched {url}")            
                  
            comments = soup.find_all(string=lambda text: isinstance(text, Comment)) # find it in the comments

            ##### get general info

            scorebox_meta_div = soup.find('div', {'class': 'scorebox_meta'})
            if scorebox_meta_div:
               # print(f"Found scorebox_meta div for {game_id}")
               
               # Get all text content with line breaks
               full_text = scorebox_meta_div.get_text(separator='\n')
               
               # Initialize variables
               date = None
               
               # Extract date - first line is usually the date
               lines = [line.strip() for line in full_text.split('\n') if line.strip()]
               
               date = lines[0]
               # print(date)

            ##### get Passing, Rushing, & Receiving table
            player_offense_table = soup.find('table', {'id': 'player_offense'})
            if player_offense_table:
               # print('found')

               player_offense_tbody = player_offense_table.find('tbody')
               if player_offense_tbody:
                  # print(player_offense_tbody)
                  rows = player_offense_tbody.find_all('tr')

                  with open('player_offense_data.csv', 'a', newline='', encoding='utf-8') as player_offense_f:
                     player_offense_writer = csv.writer(player_offense_f)

                     for row in rows:
                        if not row.has_attr("class"):
                           player_name = row.find('th', {'data-stat': 'player'}).text

                           pass_cmp_cell = row.find('td', {'data-stat': 'pass_cmp'})
                           pass_cmp = pass_cmp_cell.get_text(strip=True) if pass_cmp_cell else 0
                           pass_att_cell = row.find('td', {'data-stat': 'pass_att'})
                           pass_att = pass_att_cell.get_text(strip=True) if pass_att_cell else 0
                           pass_yds_cell = row.find('td', {'data-stat': 'pass_yds'})
                           pass_yds = pass_yds_cell.get_text(strip=True) if pass_yds_cell else 0
                           pass_td_cell = row.find('td', {'data-stat': 'pass_td'})
                           pass_td = pass_td_cell.get_text(strip=True) if pass_td_cell else 0
                           pass_int_cell = row.find('td', {'data-stat': 'pass_int'})
                           pass_int = pass_int_cell.get_text(strip=True) if pass_int_cell else 0
                           pass_sacked_cell = row.find('td', {'data-stat': 'pass_sacked'})
                           pass_sacked = pass_sacked_cell.get_text(strip=True) if pass_sacked_cell else 0
                           pass_sacked_yds_cell = row.find('td', {'data-stat': 'pass_sacked_yds'})
                           pass_sacked_yds = pass_sacked_yds_cell.get_text(strip=True) if pass_sacked_yds_cell else 0
                           pass_long_cell = row.find('td', {'data-stat': 'pass_long'})
                           pass_long = pass_long_cell.get_text(strip=True) if pass_long_cell else 0
                           pass_rating_cell = row.find('td', {'data-stat': 'pass_rating'})
                           pass_rating = pass_rating_cell.get_text(strip=True) if pass_rating_cell else 0

                           rush_att_cell = row.find('td', {'data-stat': 'rush_att'})
                           rush_att = rush_att_cell.get_text(strip=True) if rush_att_cell else 0
                           rush_yds_cell = row.find('td', {'data-stat': 'rush_yds'})
                           rush_yds = rush_yds_cell.get_text(strip=True) if rush_yds_cell else 0
                           rush_td_cell = row.find('td', {'data-stat': 'rush_td'})
                           rush_td = rush_td_cell.get_text(strip=True) if rush_td_cell else 0
                           rush_long_cell = row.find('td', {'data-stat': 'rush_long'})
                           rush_long = rush_long_cell.get_text(strip=True) if rush_long_cell else 0

                           targets_cell = row.find('td', {'data-stat': 'targets'})
                           targets = targets_cell.get_text(strip=True) if targets_cell else 0
                           rec_cell = row.find('td', {'data-stat': 'rec'})
                           rec = rec_cell.get_text(strip=True) if rec_cell else 0
                           rec_yds_cell = row.find('td', {'data-stat': 'rec_yds'})
                           rec_yds = rec_yds_cell.get_text(strip=True) if rec_yds_cell else 0
                           rec_td_cell = row.find('td', {'data-stat': 'rec_td'})
                           rec_td = rec_td_cell.get_text(strip=True) if rec_td_cell else 0
                           rec_long_cell = row.find('td', {'data-stat': 'rec_long'})
                           rec_long = rec_long_cell.get_text(strip=True) if rec_long_cell else 0

                           fumbles_cell = row.find('td', {'data-stat': 'fumbles'})
                           fumbles = fumbles_cell.get_text(strip=True) if fumbles_cell else 0
                           fumbles_lost_cell = row.find('td', {'data-stat': 'fumbles_lost'})
                           fumbles_lost = fumbles_lost_cell.get_text(strip=True) if fumbles_lost_cell else 0

                           # print(rec_cell)

                           player_offense_writer.writerow([
                              game_id,
                              player_name,
                              date,
                              pass_cmp, pass_att, pass_yds, pass_td, pass_int, pass_sacked, pass_sacked_yds, pass_long, pass_rating,
                              rush_att, rush_yds, rush_td, rush_long,
                              targets, rec, rec_yds, rec_td, rec_long,
                              fumbles, fumbles_lost
                           ])

            ##### get Defense table
            for comment in comments:
               if 'player_defense' in comment:
                  # Parse the comment content as HTML
                  comment_soup = BeautifulSoup(comment, 'html.parser')
                  player_defense_table = comment_soup.find('table', {'id': 'player_defense'})
                  if player_defense_table:
                     # print("Found table in comments")
                     player_defense_tbody = player_defense_table.find('tbody')

                     rows = player_defense_tbody.find_all('tr')

                     with open('player_defense_data.csv', 'a', newline='', encoding='utf-8') as player_defense_f:
                        player_defense_writer = csv.writer(player_defense_f)

                        for row in rows:
                           # print(row)

                           if not row.has_attr("class"):
                              player_name = row.find('th', {'data-stat': 'player'}).text

                              def_int_cell = row.find('td', {'data-stat': 'def_int'})
                              def_int = def_int_cell.get_text(strip=True) if def_int_cell else 0
                              def_int_yds_cell = row.find('td', {'data-stat': 'def_int_yds'})
                              def_int_yds = def_int_yds_cell.get_text(strip=True) if def_int_yds_cell else 0
                              def_int_td_cell = row.find('td', {'data-stat': 'def_int_td'})
                              def_int_td = def_int_td_cell.get_text(strip=True) if def_int_td_cell else 0
                              def_int_long_cell = row.find('td', {'data-stat': 'def_int_long'})
                              def_int_long = def_int_long_cell.get_text(strip=True) if def_int_long_cell else 0
                              pass_defended_cell = row.find('td', {'data-stat': 'pass_defended'})
                              pass_defended = pass_defended_cell.get_text(strip=True) if pass_defended_cell else 0

                              sacks_cell = row.find('td', {'data-stat': 'sacks'})
                              sacks = sacks_cell.get_text(strip=True) if sacks_cell else 0
                              
                              tackles_combined_cell = row.find('td', {'data-stat': 'tackles_combined'})
                              tackles_combined = tackles_combined_cell.get_text(strip=True) if tackles_combined_cell else 0
                              tackles_solo_cell = row.find('td', {'data-stat': 'tackles_solo'})
                              tackles_solo = tackles_solo_cell.get_text(strip=True) if tackles_solo_cell else 0
                              tackles_assists_cell = row.find('td', {'data-stat': 'tackles_assists'})
                              tackles_assists = tackles_assists_cell.get_text(strip=True) if tackles_assists_cell else 0
                              tackles_loss_cell = row.find('td', {'data-stat': 'tackles_loss'})
                              tackles_loss = tackles_loss_cell.get_text(strip=True) if tackles_loss_cell else 0
                              qb_hits_cell = row.find('td', {'data-stat': 'qb_hits'})
                              qb_hits = qb_hits_cell.get_text(strip=True) if qb_hits_cell else 0

                              fumbles_rec_cell = row.find('td', {'data-stat': 'fumbles_rec'})
                              fumbles_rec = fumbles_rec_cell.get_text(strip=True) if fumbles_rec_cell else 0
                              fumbles_rec_yds_cell = row.find('td', {'data-stat': 'fumbles_rec_yds'})
                              fumbles_rec_yds = fumbles_rec_yds_cell.get_text(strip=True) if fumbles_rec_yds_cell else 0
                              fumbles_rec_td_cell = row.find('td', {'data-stat': 'fumbles_rec_td'})
                              fumbles_rec_td = fumbles_rec_td_cell.get_text(strip=True) if fumbles_rec_td_cell else 0
                              fumbles_forced_cell = row.find('td', {'data-stat': 'fumbles_forced'})
                              fumbles_forced = fumbles_forced_cell.get_text(strip=True) if fumbles_forced_cell else 0

                              player_defense_writer.writerow([
                                 game_id,
                                 player_name,
                                 date,
                                 def_int, def_int_yds, def_int_td, def_int_long, pass_defended,
                                 sacks,
                                 tackles_combined, tackles_solo, tackles_assists, tackles_loss, qb_hits,
                                 fumbles_rec, fumbles_rec_yds, fumbles_rec_td, fumbles_forced
                              ])

            ##### get Kick/Punt Returns table
            for comment in comments:
               if 'returns' in comment:
                  comment_soup = BeautifulSoup(comment, 'html.parser')
                  returns_table = comment_soup.find('table', {'id': 'returns'})
                  if player_defense_table:
                     # print("Found table in comments")
                     returns_tbody = returns_table.find('tbody')

                     rows = returns_tbody.find_all('tr')

                     with open('returns_data.csv', 'a', newline='', encoding='utf-8') as returns_f:
                        returns_writer = csv.writer(returns_f)

                        for row in rows:
                           # print(row)

                           if not row.has_attr("class"):
                              player_name = row.find('th', {'data-stat': 'player'}).text

                              kick_ret_cell = row.find('td', {'data-stat': 'kick_ret'})
                              kick_ret = kick_ret_cell.get_text(strip=True) if kick_ret_cell else 0
                              kick_ret_yds_cell = row.find('td', {'data-stat': 'kick_ret_yds'})
                              kick_ret_yds = kick_ret_yds_cell.get_text(strip=True) if kick_ret_yds_cell else 0
                              kick_ret_yds_per_ret_cell = row.find('td', {'data-stat': 'kick_ret_yds_per_ret'})
                              kick_ret_yds_per_ret = kick_ret_yds_per_ret_cell.get_text(strip=True) if kick_ret_yds_per_ret_cell else 0
                              kick_ret_td_cell = row.find('td', {'data-stat': 'kick_ret_td'})
                              kick_ret_td = kick_ret_td_cell.get_text(strip=True) if kick_ret_td_cell else 0
                              kick_ret_long_cell = row.find('td', {'data-stat': 'kick_ret_long'})
                              kick_ret_long = kick_ret_long_cell.get_text(strip=True) if kick_ret_long_cell else 0

                              punt_ret_cell = row.find('td', {'data-stat': 'punt_ret'})
                              punt_ret = punt_ret_cell.get_text(strip=True) if punt_ret_cell else 0
                              punt_ret_yds_cell = row.find('td', {'data-stat': 'punt_ret_yds'})
                              punt_ret_yds = punt_ret_yds_cell.get_text(strip=True) if punt_ret_yds_cell else 0
                              punt_ret_yds_per_ret_cell = row.find('td', {'data-stat': 'punt_ret_yds_per_ret'})
                              punt_ret_yds_per_ret = punt_ret_yds_per_ret_cell.get_text(strip=True) if punt_ret_yds_per_ret_cell else 0
                              punt_ret_td_cell = row.find('td', {'data-stat': 'punt_ret_td'})
                              punt_ret_td = punt_ret_td_cell.get_text(strip=True) if punt_ret_td_cell else 0
                              punt_ret_long_cell = row.find('td', {'data-stat': 'punt_ret_long'})
                              punt_ret_long = punt_ret_long_cell.get_text(strip=True) if punt_ret_long_cell else 0

                              returns_writer.writerow([
                                 game_id,
                                 player_name,
                                 date,
                                 kick_ret, kick_ret_yds, kick_ret_yds_per_ret, kick_ret_td, kick_ret_long,
                                 punt_ret, punt_ret_yds, punt_ret_yds_per_ret, punt_ret_td, punt_ret_long
                              ])

            ##### get Kicking & Punting table
            for comment in comments:
               if 'kicking' in comment:
                  comment_soup = BeautifulSoup(comment, 'html.parser')
                  kicking_table = comment_soup.find('table', {'id': 'kicking'})
                  if kicking_table:
                     # print("Found table in comments")
                     kicking_tbody = kicking_table.find('tbody')

                     rows = kicking_tbody.find_all('tr')

                     with open('kicking_data.csv', 'a', newline='', encoding='utf-8') as kicking_f:
                        kicking_writer = csv.writer(kicking_f)

                        for row in rows:
                           # print(row)

                           if not row.has_attr("class"):
                              player_name = row.find('th', {'data-stat': 'player'}).text

                              xpm_cell = row.find('td', {'data-stat': 'xpm'})
                              xpm = xpm_cell.get_text(strip=True) if xpm_cell else 0
                              xpa_cell = row.find('td', {'data-stat': 'xpa'})
                              xpa = xpa_cell.get_text(strip=True) if xpa_cell else 0

                              fgm_cell = row.find('td', {'data-stat': 'fgm'})
                              fgm = fgm_cell.get_text(strip=True) if fgm_cell else 0
                              fga_cell = row.find('td', {'data-stat': 'fga'})
                              fga = fga_cell.get_text(strip=True) if fga_cell else 0

                              punt_cell = row.find('td', {'data-stat': 'punt'})
                              punt = punt_cell.get_text(strip=True) if punt_cell else 0
                              punt_yds_cell = row.find('td', {'data-stat': 'punt_yds'})
                              punt_yds = punt_yds_cell.get_text(strip=True) if punt_yds_cell else 0
                              punt_yds_per_punt_cell = row.find('td', {'data-stat': 'punt_yds_per_punt'})
                              punt_yds_per_punt = punt_yds_per_punt_cell.get_text(strip=True) if punt_yds_per_punt_cell else 0
                              punt_long_cell = row.find('td', {'data-stat': 'punt_long'})
                              punt_long = punt_long_cell.get_text(strip=True) if punt_long_cell else 0

                              kicking_writer.writerow([
                                 game_id,
                                 player_name,
                                 date,
                                 xpm, xpa,
                                 fgm, fga,
                                 punt, punt_yds, punt_yds_per_punt, punt_long
                              ])

            ##### get Advanced Passing table
            for comment in comments:
               if 'passing_advanced' in comment:
                  comment_soup = BeautifulSoup(comment, 'html.parser')
                  passing_advanced_table = comment_soup.find('table', {'id': 'passing_advanced'})
                  if passing_advanced_table:
                     # print("Found table in com/mentsasdfasdf")
                     passing_advanced_tbody = passing_advanced_table.find('tbody')

                     rows = passing_advanced_tbody.find_all('tr')

                     with open('passing_advanced_data.csv', 'a', newline='', encoding='utf-8') as passing_advanced_f:
                        passing_advanced_writer = csv.writer(passing_advanced_f)

                        for row in rows:
                           # print(row)

                           if not row.has_attr("class"):
                              player_name = row.find('th', {'data-stat': 'player'}).text

                              pass_cmp_cell = row.find('td', {'data-stat': 'pass_cmp'})
                              pass_cmp = pass_cmp_cell.get_text(strip=True) if pass_cmp_cell else 0
                              pass_att_cell = row.find('td', {'data-stat': 'pass_att'})
                              pass_att = pass_att_cell.get_text(strip=True) if pass_att_cell else 0
                              pass_yds_cell = row.find('td', {'data-stat': 'pass_yds'})
                              pass_yds = pass_yds_cell.get_text(strip=True) if pass_yds_cell else 0

                              pass_first_down_cell = row.find('td', {'data-stat': 'pass_first_down'})
                              pass_first_down = pass_first_down_cell.get_text(strip=True) if pass_first_down_cell else 0
                              pass_first_down_pct_cell = row.find('td', {'data-stat': 'pass_first_down_pct'})
                              pass_first_down_pct = pass_first_down_pct_cell.get_text(strip=True) if pass_first_down_pct_cell else 0

                              pass_target_yds_cell = row.find('td', {'data-stat': 'pass_target_yds'})
                              pass_target_yds = pass_target_yds_cell.get_text(strip=True) if pass_target_yds_cell else 0
                              pass_tgt_yds_per_att_cell = row.find('td', {'data-stat': 'pass_tgt_yds_per_att'})
                              pass_tgt_yds_per_att = pass_tgt_yds_per_att_cell.get_text(strip=True) if pass_tgt_yds_per_att_cell else 0

                              pass_air_yds_cell = row.find('td', {'data-stat': 'pass_air_yds'})
                              pass_air_yds = pass_air_yds_cell.get_text(strip=True) if pass_air_yds_cell else 0
                              pass_air_yds_per_cmp_cell = row.find('td', {'data-stat': 'pass_air_yds_per_cmp'})
                              pass_air_yds_per_cmp = pass_air_yds_per_cmp_cell.get_text(strip=True) if pass_air_yds_per_cmp_cell else 0
                              pass_air_yds_per_att_cell = row.find('td', {'data-stat': 'pass_air_yds_per_att'})
                              pass_air_yds_per_att = pass_air_yds_per_att_cell.get_text(strip=True) if pass_air_yds_per_att_cell else 0

                              pass_yac_cell = row.find('td', {'data-stat': 'pass_yac'})
                              pass_yac = pass_yac_cell.get_text(strip=True) if pass_yac_cell else 0
                              pass_yac_per_cmp_cell = row.find('td', {'data-stat': 'pass_yac_per_cmp'})
                              pass_yac_per_cmp = pass_yac_per_cmp_cell.get_text(strip=True) if pass_yac_per_cmp_cell else 0

                              pass_drops_cell = row.find('td', {'data-stat': 'pass_drops'})
                              pass_drops = pass_drops_cell.get_text(strip=True) if pass_drops_cell else 0
                              pass_drop_pct_cell = row.find('td', {'data-stat': 'pass_drop_pct'})
                              pass_drop_pct = pass_drop_pct_cell.get_text(strip=True) if pass_drop_pct_cell else 0

                              pass_poor_throws_cell = row.find('td', {'data-stat': 'pass_poor_throws'})
                              pass_poor_throws = pass_poor_throws_cell.get_text(strip=True) if pass_poor_throws_cell else 0
                              pass_poor_throw_pct_cell = row.find('td', {'data-stat': 'pass_poor_throw_pct'})
                              pass_poor_throw_pct = pass_poor_throw_pct_cell.get_text(strip=True) if pass_poor_throw_pct_cell else 0

                              pass_sacked_cell = row.find('td', {'data-stat': 'pass_sacked'})
                              pass_sacked = pass_sacked_cell.get_text(strip=True) if pass_sacked_cell else 0
                              pass_blitzed_cell = row.find('td', {'data-stat': 'pass_blitzed'})
                              pass_blitzed = pass_blitzed_cell.get_text(strip=True) if pass_blitzed_cell else 0
                              pass_hurried_cell = row.find('td', {'data-stat': 'pass_hurried'})
                              pass_hurried = pass_hurried_cell.get_text(strip=True) if pass_hurried_cell else 0
                              pass_hits_cell = row.find('td', {'data-stat': 'pass_hits'})
                              pass_hits = pass_hits_cell.get_text(strip=True) if pass_hits_cell else 0
                              pass_pressured_cell = row.find('td', {'data-stat': 'pass_pressured'})
                              pass_pressured = pass_pressured_cell.get_text(strip=True) if pass_pressured_cell else 0
                              pass_pressured_pct_cell = row.find('td', {'data-stat': 'pass_pressured_pct'})
                              pass_pressured_pct = pass_pressured_pct_cell.get_text(strip=True) if pass_pressured_pct_cell else 0

                              rush_scrambles_cell = row.find('td', {'data-stat': 'rush_scrambles'})
                              rush_scrambles = rush_scrambles_cell.get_text(strip=True) if rush_scrambles_cell else 0
                              rush_scrambles_yds_per_att_cell = row.find('td', {'data-stat': 'rush_scrambles_yds_per_att'})
                              rush_scrambles_yds_per_att = rush_scrambles_yds_per_att_cell.get_text(strip=True) if rush_scrambles_yds_per_att_cell else 0

                              passing_advanced_writer.writerow([
                                 game_id,
                                 player_name,
                                 date,
                                 pass_cmp, pass_att, pass_yds,
                                 pass_first_down, pass_first_down_pct,
                                 pass_target_yds, pass_tgt_yds_per_att,
                                 pass_air_yds, pass_air_yds_per_cmp, pass_air_yds_per_att,
                                 pass_yac, pass_yac_per_cmp,
                                 pass_drops, pass_drop_pct,
                                 pass_poor_throws, pass_poor_throw_pct,
                                 pass_sacked, pass_blitzed, pass_hurried, pass_hits, pass_pressured, pass_pressured_pct,
                                 rush_scrambles, rush_scrambles_yds_per_att
                              ])

            ##### get Advanced Rushing table
            for comment in comments:
               if 'rushing_advanced' in comment:
                  comment_soup = BeautifulSoup(comment, 'html.parser')
                  rushing_advanced_table = comment_soup.find('table', {'id': 'rushing_advanced'})
                  if rushing_advanced_table:
                     # print("Found table in com/mentsasdfasdf")
                     rushing_advanced_tbody = rushing_advanced_table.find('tbody')

                     rows = rushing_advanced_tbody.find_all('tr')

                     with open('rushing_advanced_data.csv', 'a', newline='', encoding='utf-8') as rushing_advanced_f:
                        rushing_advanced_writer = csv.writer(rushing_advanced_f)

                        for row in rows:
                           # print(row)

                           if not row.has_attr("class"):
                              player_name = row.find('th', {'data-stat': 'player'}).text

                              rush_att_cell = row.find('td', {'data-stat': 'rush_att'})
                              rush_att = rush_att_cell.get_text(strip=True) if rush_att_cell else 0
                              rush_yds_cell = row.find('td', {'data-stat': 'rush_yds'})
                              rush_yds = rush_yds_cell.get_text(strip=True) if rush_yds_cell else 0
                              rush_td_cell = row.find('td', {'data-stat': 'rush_td'})
                              rush_td = rush_td_cell.get_text(strip=True) if rush_td_cell else 0

                              rush_first_down_cell = row.find('td', {'data-stat': 'rush_first_down'})
                              rush_first_down = rush_first_down_cell.get_text(strip=True) if rush_first_down_cell else 0

                              rush_yds_before_contact_cell = row.find('td', {'data-stat': 'rush_yds_before_contact'})
                              rush_yds_before_contact = rush_yds_before_contact_cell.get_text(strip=True) if rush_yds_before_contact_cell else 0
                              rush_yds_bc_per_rush_cell = row.find('td', {'data-stat': 'rush_yds_bc_per_rush'})
                              rush_yds_bc_per_rush = rush_yds_bc_per_rush_cell.get_text(strip=True) if rush_yds_bc_per_rush_cell else 0

                              rush_yac_cell = row.find('td', {'data-stat': 'rush_yac'})
                              rush_yac = rush_yac_cell.get_text(strip=True) if rush_yac_cell else 0
                              rush_yac_per_rush_cell = row.find('td', {'data-stat': 'rush_yac_per_rush'})
                              rush_yac_per_rush = rush_yac_per_rush_cell.get_text(strip=True) if rush_yac_per_rush_cell else 0

                              rush_broken_tackles_cell = row.find('td', {'data-stat': 'rush_broken_tackles'})
                              rush_broken_tackles = rush_broken_tackles_cell.get_text(strip=True) if rush_broken_tackles_cell else 0
                              rush_broken_tackles_per_rush_cell = row.find('td', {'data-stat': 'rush_broken_tackles_per_rush'})
                              rush_broken_tackles_per_rush = rush_broken_tackles_per_rush_cell.get_text(strip=True) if rush_broken_tackles_per_rush_cell else 0

                              rushing_advanced_writer.writerow([
                                 game_id,
                                 player_name,
                                 date,
                                 rush_att, rush_yds, rush_td,
                                 rush_first_down,
                                 rush_yds_before_contact, rush_yds_bc_per_rush,
                                 rush_yac, rush_yac_per_rush,
                                 rush_broken_tackles, rush_broken_tackles_per_rush
                              ])

            ##### get Advanced Receiving table
            for comment in comments:
               if 'receiving_advanced' in comment: 
                  comment_soup = BeautifulSoup(comment, 'html.parser')
                  receiving_advanced_table = comment_soup.find('table', {'id': 'receiving_advanced'})  
                  if receiving_advanced_table:
                        # print("Found receiving table in comments")
                        receiving_advanced_tbody = receiving_advanced_table.find('tbody')

                        rows = receiving_advanced_tbody.find_all('tr')

                        with open('receiving_advanced_data.csv', 'a', newline='', encoding='utf-8') as receiving_advanced_f:  
                           receiving_advanced_writer = csv.writer(receiving_advanced_f)  

                           for row in rows:
                              # print(row)

                              if not row.has_attr("class"):
                                    player_name = row.find('th', {'data-stat': 'player'}).text

                                    targets_cell = row.find('td', {'data-stat': 'targets'})
                                    targets = targets_cell.get_text(strip=True) if targets_cell else 0                                 
                                    rec_cell = row.find('td', {'data-stat': 'rec'})
                                    rec = rec_cell.get_text(strip=True) if rec_cell else 0                                   
                                    rec_yds_cell = row.find('td', {'data-stat': 'rec_yds'})
                                    rec_yds = rec_yds_cell.get_text(strip=True) if rec_yds_cell else 0                                  
                                    rec_td_cell = row.find('td', {'data-stat': 'rec_td'})
                                    rec_td = rec_td_cell.get_text(strip=True) if rec_td_cell else 0

                                    rec_first_down_cell = row.find('td', {'data-stat': 'rec_first_down'})
                                    rec_first_down = rec_first_down_cell.get_text(strip=True) if rec_first_down_cell else 0

                                    rec_air_yds_cell = row.find('td', {'data-stat': 'rec_air_yds'})
                                    rec_air_yds = rec_air_yds_cell.get_text(strip=True) if rec_air_yds_cell else 0
                                    rec_air_yds_per_rec_cell = row.find('td', {'data-stat': 'rec_air_yds_per_rec'})
                                    rec_air_yds_per_rec = rec_air_yds_per_rec_cell.get_text(strip=True) if rec_air_yds_per_rec_cell else 0

                                    rec_yac_cell = row.find('td', {'data-stat': 'rec_yac'})
                                    rec_yac = rec_yac_cell.get_text(strip=True) if rec_yac_cell else 0
                                    rec_yac_per_rec_cell = row.find('td', {'data-stat': 'rec_yac_per_rec'})
                                    rec_yac_per_rec = rec_yac_per_rec_cell.get_text(strip=True) if rec_yac_per_rec_cell else 0

                                    rec_adot_cell = row.find('td', {'data-stat': 'rec_adot'})
                                    rec_adot = rec_adot_cell.get_text(strip=True) if rec_adot_cell else 0

                                    rec_broken_tackles_cell = row.find('td', {'data-stat': 'rec_broken_tackles'})
                                    rec_broken_tackles = rec_broken_tackles_cell.get_text(strip=True) if rec_broken_tackles_cell else 0
                                    rec_broken_tackles_per_rec_cell = row.find('td', {'data-stat': 'rec_broken_tackles_per_rec'})
                                    rec_broken_tackles_per_rec = rec_broken_tackles_per_rec_cell.get_text(strip=True) if rec_broken_tackles_per_rec_cell else 0

                                    rec_drops_cell = row.find('td', {'data-stat': 'rec_drops'})
                                    rec_drops = rec_drops_cell.get_text(strip=True) if rec_drops_cell else 0
                                    rec_drop_pct_cell = row.find('td', {'data-stat': 'rec_drop_pct'})
                                    rec_drop_pct = rec_drop_pct_cell.get_text(strip=True) if rec_drop_pct_cell else 0

                                    rec_target_int_cell = row.find('td', {'data-stat': 'rec_target_int'})
                                    rec_target_int = rec_target_int_cell.get_text(strip=True) if rec_target_int_cell else 0

                                    rec_pass_rating_cell = row.find('td', {'data-stat': 'rec_pass_rating'})
                                    rec_pass_rating = rec_pass_rating_cell.get_text(strip=True) if rec_pass_rating_cell else 0

                                    receiving_advanced_writer.writerow([
                                       game_id,
                                       player_name,
                                       date,
                                       targets, rec, rec_yds, rec_td,
                                       rec_first_down,
                                       rec_air_yds, rec_air_yds_per_rec,
                                       rec_yac, rec_yac_per_rec,
                                       rec_adot,
                                       rec_broken_tackles, rec_broken_tackles_per_rec,
                                       rec_drops, rec_drop_pct,
                                       rec_target_int,
                                       rec_pass_rating
                                    ])

            ##### get Advanced Defense table
            for comment in comments:
               if 'defense_advanced' in comment:
                  comment_soup = BeautifulSoup(comment, 'html.parser')
                  defense_advanced_table = comment_soup.find('table', {'id': 'defense_advanced'})
                  if defense_advanced_table:
                        # print("Found defensive table in comments")
                        defense_advanced_tbody = defense_advanced_table.find('tbody')

                        rows = defense_advanced_tbody.find_all('tr')

                        with open('defense_advanced_data.csv', 'a', newline='', encoding='utf-8') as defense_advanced_f:
                           defense_advanced_writer = csv.writer(defense_advanced_f)

                           for row in rows:
                              # print(row)

                              if not row.has_attr("class"):
                                    player_name = row.find('th', {'data-stat': 'player'}).text

                                    def_int_cell = row.find('td', {'data-stat': 'def_int'})
                                    def_int = def_int_cell.get_text(strip=True) if def_int_cell else 0

                                    def_targets_cell = row.find('td', {'data-stat': 'def_targets'})
                                    def_targets = def_targets_cell.get_text(strip=True) if def_targets_cell else 0
                                    def_cmp_cell = row.find('td', {'data-stat': 'def_cmp'})
                                    def_cmp = def_cmp_cell.get_text(strip=True) if def_cmp_cell else 0
                                    def_cmp_perc_cell = row.find('td', {'data-stat': 'def_cmp_perc'})
                                    def_cmp_perc = def_cmp_perc_cell.get_text(strip=True) if def_cmp_perc_cell else 0
                                    def_cmp_yds_cell = row.find('td', {'data-stat': 'def_cmp_yds'})
                                    def_cmp_yds = def_cmp_yds_cell.get_text(strip=True) if def_cmp_yds_cell else 0
                                    def_yds_per_cmp_cell = row.find('td', {'data-stat': 'def_yds_per_cmp'})
                                    def_yds_per_cmp = def_yds_per_cmp_cell.get_text(strip=True) if def_yds_per_cmp_cell else 0
                                    def_yds_per_target_cell = row.find('td', {'data-stat': 'def_yds_per_target'})
                                    def_yds_per_target = def_yds_per_target_cell.get_text(strip=True) if def_yds_per_target_cell else 0
                                    def_cmp_td_cell = row.find('td', {'data-stat': 'def_cmp_td'})
                                    def_cmp_td = def_cmp_td_cell.get_text(strip=True) if def_cmp_td_cell else 0

                                    def_pass_rating_cell = row.find('td', {'data-stat': 'def_pass_rating'})
                                    def_pass_rating = def_pass_rating_cell.get_text(strip=True) if def_pass_rating_cell else 0

                                    def_tgt_yds_per_att_cell = row.find('td', {'data-stat': 'def_tgt_yds_per_att'})
                                    def_tgt_yds_per_att = def_tgt_yds_per_att_cell.get_text(strip=True) if def_tgt_yds_per_att_cell else 0
                                    def_air_yds_cell = row.find('td', {'data-stat': 'def_air_yds'})
                                    def_air_yds = def_air_yds_cell.get_text(strip=True) if def_air_yds_cell else 0
                                    def_yac_cell = row.find('td', {'data-stat': 'def_yac'})
                                    def_yac = def_yac_cell.get_text(strip=True) if def_yac_cell else 0

                                    blitzes_cell = row.find('td', {'data-stat': 'blitzes'})
                                    blitzes = blitzes_cell.get_text(strip=True) if blitzes_cell else 0
                                    qb_hurry_cell = row.find('td', {'data-stat': 'qb_hurry'})
                                    qb_hurry = qb_hurry_cell.get_text(strip=True) if qb_hurry_cell else 0
                                    qb_knockdown_cell = row.find('td', {'data-stat': 'qb_knockdown'})
                                    qb_knockdown = qb_knockdown_cell.get_text(strip=True) if qb_knockdown_cell else 0
                                    sacks_cell = row.find('td', {'data-stat': 'sacks'})
                                    sacks = sacks_cell.get_text(strip=True) if sacks_cell else 0
                                    pressures_cell = row.find('td', {'data-stat': 'pressures'})
                                    pressures = pressures_cell.get_text(strip=True) if pressures_cell else 0

                                    tackles_combined_cell = row.find('td', {'data-stat': 'tackles_combined'})
                                    tackles_combined = tackles_combined_cell.get_text(strip=True) if tackles_combined_cell else 0
                                    tackles_missed_cell = row.find('td', {'data-stat': 'tackles_missed'})
                                    tackles_missed = tackles_missed_cell.get_text(strip=True) if tackles_missed_cell else 0
                                    tackles_missed_pct_cell = row.find('td', {'data-stat': 'tackles_missed_pct'})
                                    tackles_missed_pct = tackles_missed_pct_cell.get_text(strip=True) if tackles_missed_pct_cell else 0

                                    defense_advanced_writer.writerow([
                                       game_id,
                                       player_name,
                                       date,
                                       def_int,
                                       def_targets, def_cmp, def_cmp_perc, def_cmp_yds, 
                                       def_yds_per_cmp, def_yds_per_target, def_cmp_td,
                                       def_pass_rating,
                                       def_tgt_yds_per_att, def_air_yds, def_yac,
                                       blitzes, qb_hurry, qb_knockdown, sacks, pressures,
                                       tackles_combined, tackles_missed, tackles_missed_pct
                                    ])

            ##### get Starters
            for comment in comments:
               if 'home_starters' in comment:
                  comment_soup = BeautifulSoup(comment, 'html.parser')
                  home_starters_table = comment_soup.find('table', {'id': 'home_starters'})
                  if home_starters_table:
                     # print("Found table in comments")
                     home_starters_tbody = home_starters_table.find('tbody')

                     rows = home_starters_tbody.find_all('tr')

                     with open('starters_data.csv', 'a', newline='', encoding='utf-8') as starters_f:
                        starters_writer = csv.writer(starters_f)

                        for row in rows:
                           # print(row)

                           if not row.has_attr("class"):
                              player_name = row.find('th', {'data-stat': 'player'}).text

                              starters_writer.writerow([
                                 game_id,
                                 player_name,
                                 date,
                                 True
                              ])

               if 'vis_starters' in comment:
                  comment_soup = BeautifulSoup(comment, 'html.parser')
                  vis_starters_table = comment_soup.find('table', {'id': 'vis_starters'})
                  if home_starters_table:
                     # print("Found table in comments")
                     vis_starters_tbody = vis_starters_table.find('tbody')

                     rows = vis_starters_tbody.find_all('tr')

                     with open('starters_data.csv', 'a', newline='', encoding='utf-8') as starters_f:
                        starters_writer = csv.writer(starters_f)

                        for row in rows:
                           # print(row)

                           if not row.has_attr("class"):
                              player_name = row.find('th', {'data-stat': 'player'}).text

                              starters_writer.writerow([
                                 game_id,
                                 player_name,
                                 date,
                                 True
                              ])
            
         elif response.status_code == 429:
            # time.sleep(random.uniform(2, 4))
            print("429 error at " + game_id)
            game_id_429 = game_id
            break

         else:
            print(f"Failed with status: {response.status_code}")
            failed_ids.append(game_id)
               
      except Exception as e:
         print(f"Error: {e}")
         failed_ids.append(game_id)

      # with open('player_snap_data.csv', 'a', newline='', encoding='utf-8') as f:
      #       writer = csv.writer(f)

   
      # break

get_data(game_id_list)
# get_data(['202509040phi'])
   

Processing: https://www.pro-football-reference.com/boxscores/201609080den.htm
Status code: 200
Successfully fetched https://www.pro-football-reference.com/boxscores/201609080den.htm
Processing: https://www.pro-football-reference.com/boxscores/201609110rav.htm
Status code: 200
Successfully fetched https://www.pro-football-reference.com/boxscores/201609110rav.htm
Processing: https://www.pro-football-reference.com/boxscores/201609110jax.htm
Status code: 200
Successfully fetched https://www.pro-football-reference.com/boxscores/201609110jax.htm
Processing: https://www.pro-football-reference.com/boxscores/201609110atl.htm
Status code: 200
Successfully fetched https://www.pro-football-reference.com/boxscores/201609110atl.htm
Processing: https://www.pro-football-reference.com/boxscores/201609110htx.htm
Status code: 200
Successfully fetched https://www.pro-football-reference.com/boxscores/201609110htx.htm
Processing: https://www.pro-football-reference.com/boxscores/201609110nyj.htm
Status code:

In [75]:
# combine the csvs

# all snap data
player_snap_df = pd.read_csv('player_snap_data.csv')

player_offense_df = pd.read_csv('player_offense_data.csv')
player_defense_df = pd.read_csv('player_defense_data.csv')
kicking_df = pd.read_csv('kicking_data.csv')
returns_df = pd.read_csv('returns_data.csv')
passing_advanced_df = pd.read_csv('passing_advanced_data.csv')
rushing_advanced_df = pd.read_csv('rushing_advanced_data.csv')
receiving_advanced_df = pd.read_csv('receiving_advanced_data.csv')
defense_advanced_df = pd.read_csv('defense_advanced_data.csv')


In [77]:
# defense_advanced_df

from functools import reduce

# Create list of all DataFrames
all_dfs = [
    player_snap_df,
    player_offense_df,
    player_defense_df,
    kicking_df,
    returns_df,
    passing_advanced_df,
    rushing_advanced_df,
    receiving_advanced_df,
    defense_advanced_df
]

# Define merge function
def merge_dfs(left, right):
    return pd.merge(left, right, on=['game_id', 'player_name', 'date'], how='left')

# Reduce all DataFrames
combined_df = reduce(merge_dfs, all_dfs)

combined_df.to_csv('combined_data.csv', index=False)

In [94]:
combined_data = pd.read_csv('combined_data.csv')
# combined_data

for column in combined_data.columns:
    print(combined_data[column].unique())

C:\Users\bchm5\AppData\Local\Temp\ipykernel_18924\1775109482.py:1: DtypeWarning: Columns (13,81,83,89,121,137) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_data = pd.read_csv('combined_data.csv')


['201609080den' '201609110rav' '201609110jax' ... '202101240gnb'
 '202101240kan' '202102070tam']
['Michael Schofield' 'Matt Paradis' 'Russell Okung' ...
 'Cedrick Lattimore' 'Blake Hance' 'Javon Hagan']
['Thursday Sep 8, 2016' 'Sunday Sep 11, 2016' 'Monday Sep 12, 2016'
 'Thursday Sep 15, 2016' 'Sunday Sep 18, 2016' 'Monday Sep 19, 2016'
 'Thursday Sep 22, 2016' 'Sunday Sep 25, 2016' 'Monday Sep 26, 2016'
 'Thursday Sep 29, 2016' 'Sunday Oct 2, 2016' 'Monday Oct 3, 2016'
 'Thursday Oct 6, 2016' 'Sunday Oct 9, 2016' 'Monday Oct 10, 2016'
 'Thursday Oct 13, 2016' 'Sunday Oct 16, 2016' 'Monday Oct 17, 2016'
 'Thursday Oct 20, 2016' 'Sunday Oct 23, 2016' 'Monday Oct 24, 2016'
 'Thursday Oct 27, 2016' 'Sunday Oct 30, 2016' 'Monday Oct 31, 2016'
 'Thursday Nov 3, 2016' 'Sunday Nov 6, 2016' 'Monday Nov 7, 2016'
 'Thursday Nov 10, 2016' 'Sunday Nov 13, 2016' 'Monday Nov 14, 2016'
 'Thursday Nov 17, 2016' 'Sunday Nov 20, 2016' 'Monday Nov 21, 2016'
 'Thursday Nov 24, 2016' 'Sunday Nov 27, 2016'

In [95]:
combined_data['wind'] = combined_data['wind'].replace(' wind', np.nan)

combined_data['wind'].unique()

array(['0 mph', '8 mph', '3 mph', nan, '7 mph', '4 mph', '9 mph', '5 mph',
       '2 mph', '1 mph', '6 mph'], dtype=object)

In [99]:
# US vs non US stadiums - UPDATED
us_stadiums = [
    'Raymond James Stadium',  # Tampa, FL
    'Mercedes-Benz Stadium',  # Atlanta, GA
    'Highmark Stadium',  # Buffalo, NY
    'Bank of America Stadium',  # Charlotte, NC
    'Paul Brown Stadium',  # Cincinnati, OH
    'Lucas Oil Stadium',  # Indianapolis, IN
    'Nissan Stadium',  # Nashville, TN
    'Ford Field',  # Detroit, MI
    'NRG Stadium',  # Houston, TX
    'FedExField',  # Landover, MD
    'GEHA Field at Arrowhead Stadium',  # Kansas City, MO
    'MetLife Stadium',  # East Rutherford, NJ
    'TIAA Bank Stadium',  # Jacksonville, FL
    'Gillette Stadium',  # Foxborough, MA
    'SoFi Stadium',  # Inglewood, CA
    'Allegiant Stadium',  # Las Vegas, NV
    'Hard Rock Stadium',  # Miami Gardens, FL
    'Soldier Field',  # Chicago, IL
    'FirstEnergy Stadium',  # Cleveland, OH
    'Lincoln Financial Field',  # Philadelphia, PA
    'Heinz Field',  # Pittsburgh, PA
    'State Farm Stadium',  # Glendale, AZ
    'Lumen Field',  # Seattle, WA
    'M&T Bank Stadium',  # Baltimore, MD
    'Lambeau Field',  # Green Bay, WI
    'Empower Field at Mile High',  # Denver, CO
    'U.S. Bank Stadium',  # Minneapolis, MN
    "Levi's Stadium",  # Santa Clara, CA
    'AT&T Stadium',  # Arlington, TX
    'Caesars Superdome',  # New Orleans, LA
    'Paycor Stadium',  # Cincinnati, OH (renamed from Paul Brown)
    'Acrisure Stadium',  # Pittsburgh, PA (renamed from Heinz)
    'Cleveland Browns Stadium',  # Cleveland, OH (same as FirstEnergy)
    'EverBank Stadium',  # Jacksonville, FL (same as TIAA Bank)
    'Huntington Bank Field',  # Chicago, IL (part of Soldier Field complex)
    'Northwest Stadium',  # Maryland
    'Sports Authority Field at Mile High',  # Denver, CO (old name for Empower Field)
    'EverBank Field',  # Jacksonville, FL (old name for EverBank/TIAA Bank)
    'Georgia Dome',  # Atlanta, GA (old, replaced by Mercedes-Benz)
    'Arrowhead Stadium',  # Kansas City, MO (old name for GEHA Field)
    'Mercedes-Benz Superdome',  # New Orleans, LA (old name for Caesars Superdome)
    'CenturyLink Field',  # Seattle, WA (old name for Lumen Field)
    'University of Phoenix Stadium',  # Glendale, AZ (old name for State Farm)
    'New Era Field',  # Buffalo, NY (old name for Highmark Stadium)
    'Los Angeles Memorial Coliseum',  # Los Angeles, CA
    'Oakland-Alameda County Coliseum',  # Oakland, CA
    'Qualcomm Stadium',  # San Diego, CA
    'StubHub Center',  # Carson, CA
    'Ring Central Coliseum',  # Oakland, CA (alternative name for Oakland-Alameda)
    'Bills Stadium'  # Buffalo, NY (alternate name for Highmark/New Era)
]

# Non-US stadiums - UPDATED
non_us_stadiums = [
    'Tottenham Stadium',  # London, UK
    'Wembley Stadium',  # London, UK
    'Allianz Arena',  # Munich, Germany
    'Azteca Stadium',  # Mexico City, Mexico
    'Deutsche Bank Park',  # Frankfurt, Germany
    'Neo Quimica Arena',  # São Paulo, Brazil
    'Croke Park',  # Dublin, Ireland
    'Olympiastadion',  # Berlin, Germany
    'Estadio Santiago Bernabéu',  # Madrid, Spain
    'Twickenham Stadium'  # London, UK
]

# stadiums by time zone - UPDATED:

# Brasilia Time Zone
GMT_minus_3 = [
    'Neo Quimica Arena',  # São Paulo, Brazil
]

# Eastern Time Zone (ET) - GMT-5 - UPDATED
GMT_minus_5 = [
    'Raymond James Stadium',  # Tampa, FL
    'Mercedes-Benz Stadium',  # Atlanta, GA
    'Highmark Stadium',  # Buffalo, NY
    'Bank of America Stadium',  # Charlotte, NC
    'Paul Brown Stadium',  # Cincinnati, OH
    'Nissan Stadium',  # Nashville, TN
    'Northwest Stadium', # Maryland
    'Ford Field',  # Detroit, MI
    'FedExField',  # Landover, MD
    'MetLife Stadium',  # East Rutherford, NJ
    'TIAA Bank Stadium',  # Jacksonville, FL
    'Gillette Stadium',  # Foxborough, MA
    'Hard Rock Stadium',  # Miami Gardens, FL
    'FirstEnergy Stadium',  # Cleveland, OH
    'Lincoln Financial Field',  # Philadelphia, PA
    'Heinz Field',  # Pittsburgh, PA
    'M&T Bank Stadium',  # Baltimore, MD
    'Paycor Stadium',  # Cincinnati, OH
    'Acrisure Stadium',  # Pittsburgh, PA
    'Cleveland Browns Stadium',  # Cleveland, OH
    'EverBank Stadium',  # Jacksonville, FL
    'Lucas Oil Stadium',  # Indianapolis, IN
    'Georgia Dome',  # Atlanta, GA (Eastern Time)
    'New Era Field',  # Buffalo, NY (Eastern Time)
    'EverBank Field',  # Jacksonville, FL (Eastern Time)
    'Bills Stadium'  # Buffalo, NY (Eastern Time)
]

# Central Time Zone (CT) - GMT-6 - UPDATED
GMT_minus_6 = [ 
    'Lambeau Field', # Green Bay, WI
    'NRG Stadium',  # Houston, TX
    'GEHA Field at Arrowhead Stadium',  # Kansas City, MO
    'AT&T Stadium',  # Arlington, TX
    'Soldier Field',  # Chicago, IL
    'Huntington Bank Field',  # Chicago, IL
    'U.S. Bank Stadium',  # Minneapolis, MN
    'Caesars Superdome',  # New Orleans, LA
    'Azteca Stadium',  # Mexico City
    'Arrowhead Stadium',  # Kansas City, MO (Central Time)
    'Mercedes-Benz Superdome'  # New Orleans, LA (Central Time)
]

# Mountain Time Zone (MT) - GMT-7 - UPDATED
GMT_minus_7 = [
    'State Farm Stadium',  # Glendale, AZ
    'Empower Field at Mile High',  # Denver, CO
    'Sports Authority Field at Mile High',  # Denver, CO (Mountain Time)
    'University of Phoenix Stadium'  # Glendale, AZ (Mountain Time)
]

# Pacific Time Zone (PT) - GMT-8 - UPDATED
GMT_minus_8 = [
    'SoFi Stadium',  # Inglewood, CA
    'Lumen Field',  # Seattle, WA
    "Levi's Stadium",  # Santa Clara, CA
    'Allegiant Stadium',  # Las Vegas, NV
    'CenturyLink Field',  # Seattle, WA (Pacific Time)
    'Los Angeles Memorial Coliseum',  # Los Angeles, CA (Pacific Time)
    'Oakland-Alameda County Coliseum',  # Oakland, CA (Pacific Time)
    'Qualcomm Stadium',  # San Diego, CA (Pacific Time)
    'StubHub Center',  # Carson, CA (Pacific Time)
    'Ring Central Coliseum'  # Oakland, CA (Pacific Time)
]

# British Time Zone - GMT - UPDATED
GMT = [
    'Croke Park',  # Dublin, Ireland
    'Tottenham Stadium',  # London, UK
    'Wembley Stadium',  # London, UK
    # ADDED MISSING STADIUM:
    'Twickenham Stadium'  # London, UK (GMT/BST)
]

# Central European Time Zone - GMT+1 - UPDATED (NO CHANGE)
GMT_plus_1 = [
    'Allianz Arena',  # Munich, Germany
    'Deutsche Bank Park',  # Frankfurt, Germany
    'Olympiastadion',  # Berlin, Germany
    'Estadio Santiago Bernabéu',  # Madrid, Spain
]

# Team time zone mapping (based on home city)
team_timezones = {
    # Eastern Time (ET) - GMT-5
    'Atlanta Falcons': -5,  # Atlanta, GA
    'Baltimore Ravens': -5,  # Baltimore, MD
    'Buffalo Bills': -5,  # Buffalo, NY
    'Carolina Panthers': -5,  # Charlotte, NC
    'Cincinnati Bengals': -5,  # Cincinnati, OH
    'Cleveland Browns': -5,  # Cleveland, OH
    'Detroit Lions': -5,  # Detroit, MI
    'Indianapolis Colts': -5,  # Indianapolis, IN
    'Jacksonville Jaguars': -5,  # Jacksonville, FL
    'Miami Dolphins': -5,  # Miami Gardens, FL
    'New England Patriots': -5,  # Foxborough, MA
    'New York Giants': -5,  # East Rutherford, NJ
    'New York Jets': -5,  # East Rutherford, NJ
    'Philadelphia Eagles': -5,  # Philadelphia, PA
    'Pittsburgh Steelers': -5,  # Pittsburgh, PA
    'Tampa Bay Buccaneers': -5,  # Tampa, FL
    'Tennessee Titans': -5,  # Nashville, TN
    'Washington Redskins': -5,  # Landover, MD
    'Washington Football Team': -5,  # Landover, MD
    
    # Central Time (CT) - GMT-6
    'Chicago Bears': -6,  # Chicago, IL
    'Dallas Cowboys': -6,  # Arlington, TX
    'Green Bay Packers': -6,  # Green Bay, WI
    'Houston Texans': -6,  # Houston, TX
    'Kansas City Chiefs': -6,  # Kansas City, MO
    'Minnesota Vikings': -6,  # Minneapolis, MN
    'New Orleans Saints': -6,  # New Orleans, LA
    
    # Mountain Time (MT) - GMT-7
    'Arizona Cardinals': -7,  # Glendale, AZ
    'Denver Broncos': -7,  # Denver, CO
    
    # Pacific Time (PT) - GMT-8
    'Las Vegas Raiders': -8,  # Las Vegas, NV
    'Los Angeles Chargers': -8,  # Inglewood, CA
    'Los Angeles Rams': -8,  # Inglewood, CA
    'Oakland Raiders': -8,  # Oakland, CA
    'San Diego Chargers': -8,  # San Diego, CA
    'San Francisco 49ers': -8,  # Santa Clara, CA
    'Seattle Seahawks': -8,  # Seattle, WA
}

# If you also want time zone abbreviations
team_timezones_abbr = {
    # Eastern Time (ET)
    'Atlanta Falcons': 'ET',
    'Baltimore Ravens': 'ET',
    'Buffalo Bills': 'ET',
    'Carolina Panthers': 'ET',
    'Cincinnati Bengals': 'ET',
    'Cleveland Browns': 'ET',
    'Detroit Lions': 'ET',
    'Indianapolis Colts': 'ET',
    'Jacksonville Jaguars': 'ET',
    'Miami Dolphins': 'ET',
    'New England Patriots': 'ET',
    'New York Giants': 'ET',
    'New York Jets': 'ET',
    'Philadelphia Eagles': 'ET',
    'Pittsburgh Steelers': 'ET',
    'Tampa Bay Buccaneers': 'ET',
    'Tennessee Titans': 'ET',
    'Washington Redskins': 'ET',
    'Washington Football Team': 'ET',
    
    # Central Time (CT)
    'Chicago Bears': 'CT',
    'Dallas Cowboys': 'CT',
    'Green Bay Packers': 'CT',
    'Houston Texans': 'CT',
    'Kansas City Chiefs': 'CT',
    'Minnesota Vikings': 'CT',
    'New Orleans Saints': 'CT',
    
    # Mountain Time (MT)
    'Arizona Cardinals': 'MT',
    'Denver Broncos': 'MT',
    
    # Pacific Time (PT)
    'Las Vegas Raiders': 'PT',
    'Los Angeles Chargers': 'PT',
    'Los Angeles Rams': 'PT',
    'Oakland Raiders': 'PT',
    'San Diego Chargers': 'PT',
    'San Francisco 49ers': 'PT',
    'Seattle Seahawks': 'PT',
}


In [97]:
# Create dictionary with numerical time zone offsets (hours from GMT)
stadium_timezone_offsets = {}

# Add all time zone offsets
for stadium in GMT_minus_3:
    stadium_timezone_offsets[stadium] = -3  # GMT-3

for stadium in GMT_minus_5:
    stadium_timezone_offsets[stadium] = -5  # ET

for stadium in GMT_minus_6:
    stadium_timezone_offsets[stadium] = -6  # CT

for stadium in GMT_minus_7:
    stadium_timezone_offsets[stadium] = -7  # MT

for stadium in GMT_minus_8:
    stadium_timezone_offsets[stadium] = -8  # PT

for stadium in GMT:
    stadium_timezone_offsets[stadium] = 0  # GMT

for stadium in GMT_plus_1:
    stadium_timezone_offsets[stadium] = 1  # CET

print(f"Created mapping for {len(stadium_timezone_offsets)} stadiums")

# Add columns to your DataFrame
combined_data['time_zone_offset'] = combined_data['stadium'].map(stadium_timezone_offsets)
combined_data['is_international'] = combined_data['stadium'].isin(non_us_stadiums)

# Check for any stadiums not in the mapping
missing_stadiums = combined_data[combined_data['time_zone_offset'].isna()]['stadium'].unique()
if len(missing_stadiums) > 0:
    print(f"\nWarning: {len(missing_stadiums)} stadiums not found in mapping:")
    for stadium in missing_stadiums:
        print(f"  - {stadium}")
    
    # Fill missing values with NaN or a default
    combined_data['time_zone_offset'] = combined_data['time_zone_offset'].fillna(np.nan)

Created mapping for 60 stadiums


In [102]:
# Add team timezone column to your DataFrame
# Assuming you have a 'team' column in your DataFrame
combined_data['team_timezone_offset'] = combined_data['player_team'].map(team_timezones)
combined_data['team_timezone'] = combined_data['player_team'].map(team_timezones_abbr)

# Check for any teams not in the mapping
missing_teams = combined_data[combined_data['team_timezone_offset'].isna()]['player_team'].unique()
if len(missing_teams) > 0:
    print(f"Warning: {len(missing_teams)} teams not found in mapping:")
    for team in missing_teams:
        print(f"  - {team}")
    
    # Fill missing values
    combined_data['team_timezone_offset'] = combined_data['team_timezone_offset'].fillna(np.nan)
    combined_data['team_timezone'] = combined_data['team_timezone'].fillna('Unknown')

# Verify the mapping
print("\nTeam timezone distribution (numerical offset):")
print(combined_data['team_timezone_offset'].value_counts().sort_index())

print("\nTeam timezone distribution (abbreviation):")
print(combined_data['team_timezone'].value_counts())

# Show a sample
print("\nSample data with team timezone columns:")
sample_cols = ['player_team', 'team_timezone_offset', 'team_timezone']
print(combined_data[sample_cols].head(15))


Team timezone distribution (numerical offset):
team_timezone_offset
-8    18771
-7     7196
-6    26978
-5    67494
Name: count, dtype: int64

Team timezone distribution (abbreviation):
team_timezone
ET    67494
CT    26978
PT    18771
MT     7196
Name: count, dtype: int64

Sample data with team timezone columns:
       player_team  team_timezone_offset team_timezone
0   Denver Broncos                    -7            MT
1   Denver Broncos                    -7            MT
2   Denver Broncos                    -7            MT
3   Denver Broncos                    -7            MT
4   Denver Broncos                    -7            MT
5   Denver Broncos                    -7            MT
6   Denver Broncos                    -7            MT
7   Denver Broncos                    -7            MT
8   Denver Broncos                    -7            MT
9   Denver Broncos                    -7            MT
10  Denver Broncos                    -7            MT
11  Denver Broncos      

In [105]:
# Calculate the time zone difference (game location vs team's home)
combined_data['game_tz_difference'] = combined_data['time_zone_offset'] - combined_data['team_timezone_offset']

# Check the results
print("Time zone difference distribution:")
print(combined_data['game_tz_difference'].value_counts().sort_index())

print("\nDescriptive statistics for time zone difference:")
print(combined_data['game_tz_difference'].describe())

# Show a sample
print("\nSample data with time zone difference:")
sample_cols = ['stadium', 'player_team', 'time_zone_offset', 'team_timezone_offset', 'game_tz_difference']
print(combined_data[sample_cols].head(15))

Time zone difference distribution:
game_tz_difference
-3     3659
-2     3713
-1     9868
 0    84592
 1     9465
 2     3694
 3     4209
 5      707
 6      178
 7       45
 8      309
Name: count, dtype: int64

Descriptive statistics for time zone difference:
count    120439.000000
mean          0.071397
std           1.171261
min          -3.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           8.000000
Name: game_tz_difference, dtype: float64

Sample data with time zone difference:
                                stadium     player_team  time_zone_offset  \
0   Sports Authority Field at Mile High  Denver Broncos                -7   
1   Sports Authority Field at Mile High  Denver Broncos                -7   
2   Sports Authority Field at Mile High  Denver Broncos                -7   
3   Sports Authority Field at Mile High  Denver Broncos                -7   
4   Sports Authority Field at Mile High  Denver Broncos                -7   
5   Sports 

In [110]:
# Add travel direction and magnitude
def add_travel_features(df, tz_diff_col='game_tz_difference'):
    """
    Add comprehensive travel features based on timezone difference
    """
    df = df.copy()
    
    # 1. Basic travel direction
    df['travel_direction'] = np.select(
        [
            df[tz_diff_col] == 0,
            df[tz_diff_col] > 0,
            df[tz_diff_col] < 0
        ],
        ['home', 'east', 'west'],
        default='unknown'
    )
    
    # 2. Absolute timezone difference (magnitude)
    df['tz_diff_magnitude'] = df[tz_diff_col].abs()
    
    # 3. Travel category based on magnitude
    conditions = [
        df[tz_diff_col] == 0,
        df['tz_diff_magnitude'] == 1,
        df['tz_diff_magnitude'] == 2,
        df['tz_diff_magnitude'] == 3,
        df['tz_diff_magnitude'] >= 4
    ]
    
    categories = ['home', '1_hr', '2_hr', '3_hr', '4+_hr']
    df['travel_magnitude'] = np.select(conditions, categories, default='unknown')
    
    # 4. Is long travel? (3+ hours difference)
    df['is_long_travel'] = df['tz_diff_magnitude'] >= 3
    
    # 5. Detailed travel description
    def get_travel_description(row):
        tz_diff = row[tz_diff_col]
        if pd.isna(tz_diff):
            return 'unknown'
        if tz_diff == 0:
            return 'home_game'
        direction = 'east' if tz_diff > 0 else 'west'
        hours = abs(int(tz_diff))
        return f'travel_{direction}_{hours}_hours'
    
    df['travel_description'] = df.apply(get_travel_description, axis=1)
    
    return df

# Apply the function
combined_data = add_travel_features(combined_data, 'game_tz_difference')

# Analyze the results
print("Travel analysis:")
print("=" * 50)

# Direction breakdown
print("\n1. Travel Direction:")
print(combined_data['travel_direction'].value_counts().sort_index())

# Magnitude breakdown
print("\n2. Travel Magnitude:")
print(combined_data['travel_magnitude'].value_counts().sort_index())

# Long travel analysis
print("\n3. Long Travel Analysis:")
long_travel_pct = combined_data['is_long_travel'].mean() * 100
print(f"Games with long travel (3+ hours): {long_travel_pct:.1f}%")

# Most common travel scenarios
print("\n4. Most Common Travel Scenarios:")
common_scenarios = combined_data['travel_description'].value_counts().head(10)
for scenario, count in common_scenarios.items():
    pct = (count / len(combined_data)) * 100
    print(f"  {scenario}: {count} games ({pct:.1f}%)")

# Sample output
print("\n5. Sample Data:")
sample = combined_data[['player_team', 'game_tz_difference', 'travel_direction', 
                       'travel_magnitude', 'travel_description']].head(10)
print(sample)

Travel analysis:

1. Travel Direction:
travel_direction
east    18607
home    84592
west    17240
Name: count, dtype: int64

2. Travel Magnitude:
travel_magnitude
1_hr     19333
2_hr      7407
3_hr      7868
4+_hr     1239
home     84592
Name: count, dtype: int64

3. Long Travel Analysis:
Games with long travel (3+ hours): 7.6%

4. Most Common Travel Scenarios:
  home_game: 84592 games (70.2%)
  travel_west_1_hours: 9868 games (8.2%)
  travel_east_1_hours: 9465 games (7.9%)
  travel_east_3_hours: 4209 games (3.5%)
  travel_west_2_hours: 3713 games (3.1%)
  travel_east_2_hours: 3694 games (3.1%)
  travel_west_3_hours: 3659 games (3.0%)
  travel_east_5_hours: 707 games (0.6%)
  travel_east_8_hours: 309 games (0.3%)
  travel_east_6_hours: 178 games (0.1%)

5. Sample Data:
      player_team  game_tz_difference travel_direction travel_magnitude  \
0  Denver Broncos                   0             home             home   
1  Denver Broncos                   0             home             hom

In [113]:
# combined_data['travel_description'].unique()
combined_data.to_csv('player_snap_data_tz.csv')

In [115]:
player_snap_data_tz = pd.read_csv('player_snap_data_tz.csv')
# player_snap_data_tz

C:\Users\bchm5\AppData\Local\Temp\ipykernel_18924\868635974.py:1: DtypeWarning: Columns (14,82,84,90,122,138) have mixed types. Specify dtype option on import or set low_memory=False.
  player_snap_data_tz = pd.read_csv('player_snap_data_tz.csv')
